<a href="https://colab.research.google.com/github/bibowskii/LLM-T5-small-summarizer/blob/main/t5_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rouge-score nltk

In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive
import time
from transformers import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
import re
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import nltk
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

nltk.download('punkt')  # Download necessary data for BLEU calculation
drive.mount('/content/drive')

In [ ]:
myModel = T5ForConditionalGeneration.from_pretrained(
    '/content/drive/MyDrive/val/model',
    local_files_only=True
)
print(myModel)
tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')

In [ ]:


testingSet = load_dataset("olanasir/cnn-daily-mail", split="test")
validationSet = load_dataset("olanasir/cnn-daily-mail", split="validation")

#checking the data

print(testingSet[0])
print(validationSet[0])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
myModel.to(device)
print(device)

In [ ]:


def tokenize_dataset(dataset):


    inputs = tokenizer(dataset['article'], return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(dataset['highlights'], return_tensors="pt", padding="max_length", truncation=True, max_length=128)

    inputs['labels'] = targets['input_ids']
    inputs['decoder_attention_mask'] = targets['attention_mask']
    return inputs




In [ ]:
tokenized_testingSet = testingSet.map(tokenize_dataset, batched=True)
tokenized_validationSet = validationSet.map(tokenize_dataset, batched=True)

In [ ]:
tokenized_testingSet.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
tokenized_validationSet.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

In [ ]:
testingLoader = DataLoader(tokenized_testingSet, batch_size=8, shuffle= False, num_workers=2)
validationLoader = DataLoader(tokenized_validationSet, batch_size=8, shuffle= False, num_workers=2)

In [ ]:

def calculate_rouge(reference, hypothesis):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(reference, hypothesis)
  return scores

def calculate_bleu(reference, hypothesis):
  reference_tokens = nltk.word_tokenize(reference)
  hypothesis_tokens = nltk.word_tokenize(hypothesis)
  bleu_score = sentence_bleu([reference_tokens], hypothesis_tokens)
  return bleu_score

In [ ]:
def validate_model(model, validationLoader, device):
  model.eval()
  total_loss = 0
  with torch.no_grad():
    for batch in validationLoader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss
      total_loss += loss.item()
  return total_loss / len(validationLoader)

In [ ]:
def test_model(model, testingLoader, tokenizer, device):
    model.eval()  # Ensure model is in evaluation mode
    predictions = []
    all_references = []
    all_hypotheses = []

    with torch.no_grad():
        for batch in testingLoader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # Get the reference summaries

            # Generate predictions from the model
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
            predictions.append(prediction)

            # Decode reference summary
            reference = tokenizer.decode(labels[0], skip_special_tokens=True)
            all_references.append(reference)
            all_hypotheses.append(prediction)

    # Calculate ROUGE and BLEU scores for all predictions
    rouge_scores = []
    bleu_scores = []
    for ref, hyp in zip(all_references, all_hypotheses):
        rouge_scores.append(calculate_rouge(ref, hyp))
        bleu_scores.append(calculate_bleu(ref, hyp))

    # Calculate average scores
    avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"Average ROUGE-1: {avg_rouge1:.4f}")
    print(f"Average ROUGE-2: {avg_rouge2:.4f}")
    print(f"Average ROUGE-L: {avg_rougeL:.4f}")
    print(f"Average BLEU: {avg_bleu:.4f}")

    return predictions

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
if __name__ == "__main__":
  validate_model(myModel, validationLoader, device)
  test_model(myModel, testingLoader,tokenizer ,device)



In [ ]:

def summarize_text(text):
    # Prepare the input text for the model
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

    # Move tensors to the correct device (GPU or CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    myModel.to(device)

    # Generate summary
    summary_ids = myModel.generate(inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device), max_length=130, min_length=30, do_sample=False)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test the summarizer with manual text input
while True:
    text = input("Enter the text to summarize (or type 'exit' to quit): ")
    if text.lower() == 'exit':
        break
    print("\nSummary:", summarize_text(text))